In [1]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
import logging

%run token.py

updater = Updater(token=token_id, use_context=True)
dispatcher = updater.dispatcher

In [2]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',level=logging.INFO)

In [ ]:
start_text = '''
Olá!
Sou um bot de notícias e informações sobre a Covid-19!

Peça /ajuda para saber as sessões que posso mostrar!
      
      

O serviço de jornalismo do Instituto Oswaldo Cruz fez a triagem do conteúdo aqui.
Sou um bot repassador não oficial! Recebo deles e repasso para vocês.      
Não conhece o Instituto Oswaldo Cruz?
Entre em www.ioc.fiocruz.br para conhecer!
(Bot não-oficial administrado por Tiago Lubiana)

(em desenvolvimento)
'''

In [ ]:
def start(update, context):
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=start_text)
        
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [ ]:
with open('18_05_2020_p1.txt', 'r') as file_1:
        comunicado_p1 = file_1.read()
        comunicado_parseado_p1 = comunicado_p1.split("--")
        
try:
    with open('18_05_2020_p2.txt', 'r') as file_2:
            comunicado_p2 = file_2.read()
            comunicado_parseado_p2 = comunicado_p2.split("--")
except:
    comunicado_parseado_p2 = {}

In [ ]:
def get_header(section):
    raw_header = section.split("\n\n")[0]
    header = raw_header.replace("\n", "")
    header = header.replace("\U0001f97c ", "")
    header = header.replace("\U0001f9ea", "")
    return(header)

def get_section_content(section):
    raw_content = section[1:]
    return(raw_content)

def get_header_section_dict(comunicado_parseado):
    comunicado_headers = [get_header(section) for section in comunicado_parseado]
    comunicado_sections = [get_section_content(section) for section in comunicado_parseado]
    comunicado_header_to_section = dict(zip(comunicado_headers, comunicado_sections ))
    return(comunicado_header_to_section)

def get_combined_section(d1, d2, section_header,data_da_atualização):
        try:
            d1_section = d1[section_header]
            d1_section = " - Parte 1 - " + data_da_atualização + "\n\n" + d1_section
        except: 
            print(" Parte 1 ausente ou sem " + section_header)
            d1_section=""
        
        try:
            d2_section = d2[section_header]
            d2_section = " - Parte 2 - " + data_da_atualização + "\n\n" + d2_section
        except: 
            print(" Parte 2 ausente ou sem " + section_header)
            d2_section=""
            
        combined_section = d1_section + "\n\n" + d2_section  
        
        if combined_section == "" + "\n\n" + "":
            combined_section = "Seção não está presente no comunicado de hoje."
        
        return(combined_section)


def concatenatar_partes(d1, d2, possible_section_headers,data_da_atualização):

    comunicado_completo_de_hoje = {}


    for section_header in possible_section_headers:
        
        combined_section = get_combined_section(d1, d2,
                                                section_header,
                                                data_da_atualização)        
        comunicado_completo_de_hoje[section_header] = combined_section

  
    return(comunicado_completo_de_hoje)


def split_parts(text):
        texto_em_chunks = text.split('\n\n')
    
        t = (list("Parte 2" in string for string in text.split('\n\n')))
        
        part_2_beginning = [i for i, x in enumerate(t) if x][0]
    
        text_1 = "\n\n".join(texto_em_chunks[:part_2_beginning])
        text_2 = "\n\n".join(texto_em_chunks[part_2_beginning:])
    
        return(text_1, text_2)


In [ ]:
possible_section_headers= ['📄 Assembleia Mundial da Saúde ','🏥 Ministério da Saúde','🔬 Pesquisa científica', ' Diagnóstico', '📊 Epidemiologia', '💊 Tratamento', 
 '🏡 Distanciamento social', '🗂️ Política', 'Saúde Pública', '🌐 Internacional', 
 '🔍 Recortes da pandemia',  '✅ Curtas', 
 'Produzido pelo Serviço de Jornalismo e Comunicação do Instituto Oswaldo Cruz (IOC/Fiocruz)']

In [ ]:
d1 = get_header_section_dict(comunicado_parseado_p1)
d2  = get_header_section_dict(comunicado_parseado_p2)
data_da_atualização = "15 de maio de 2020"

# def concatenate_dicts(d,d1):
comunicado_header_to_section = concatenatar_partes(d1, d2, possible_section_headers,data_da_atualização)


In [ ]:
if comunicado_parseado_p1[0] != "":
    comunicado_header_to_section["Balanço"] = comunicado_parseado_p1[0].split("\n\n")[1]

try: 
    if comunicado_parseado_p2[0] != "":
        comunicado_header_to_section["Balanço"] = comunicado_parseado_p2[0].split("\n\n")[1]
except: None
    
comunicado_header_to_section["Balanço"] 

In [ ]:
comunicado_header_to_section.keys()

In [ ]:
updater.start_polling()

In [ ]:
def ajuda(update, context):
    text = '''
    O robô possui atualmente funções para os seguintes temas
    - 📈 Balanço geral de casos (/balan)
    - 🏥 Ministério da Saúde (/ministerio)
    - 💊 Tratamento (/tratamento)
    - 📊 Epidemiologia (/epid)
    - 🔬 Pesquisa científica (/pesquisa)
    - Diagnóstico (/diag)
    - 🏡 Distanciamento social (/distanciamento)
    - 🗂️ Política (/politica)
    - Saúde Pública (/saude)
    - 📄 Assembleia Mundial da Saúde (/assembleia)
    - 🌐 Internacional (/internacional)
    - 🔍 Recortes da pandemia (/recortes)
    - ✅ Curtas (/curtas)  
    '''
    context.bot.send_message(chat_id=update.effective_chat.id,
                             text=text)
    
ajuda_handler = CommandHandler('ajuda', ajuda)
dispatcher.add_handler(ajuda_handler)

In [ ]:
# Repetição de código pois não conheço tão bem o módulo telegram
# O código grita por refatoração. 


def epid(update, context):
    text = comunicado_header_to_section["📊 Epidemiologia"]
    
    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
     
    
handler_for_this_call = CommandHandler('epid', epid)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def assembleia(update, context):
    text = comunicado_header_to_section["📄 Assembleia Mundial da Saúde "]
    
    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
     
handler_for_this_call = CommandHandler('assembleia', assembleia)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def balan(update, context):
    text = comunicado_header_to_section["Balanço"]
    
    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
     
    
handler_for_this_call = CommandHandler('balan', balan)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def diag(update, context):
    text = comunicado_header_to_section[" Diagnóstico"]
    
    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
     
    
handler_for_this_call = CommandHandler('diag', diag)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def politica(update, context):
    text = comunicado_header_to_section["🗂️ Política"]
    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
     
    
handler_for_this_call = CommandHandler('politica', politica)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def internacional(update, context):
    text = comunicado_header_to_section["🌐 Internacional"]

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)

handler_for_this_call = CommandHandler('internacional', internacional)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def recortes(update, context):
    text = comunicado_header_to_section['🔍 Recortes da pandemia']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
    
handler_for_this_call = CommandHandler('recortes', recortes)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def curtas(update, context):
    text = comunicado_header_to_section['✅ Curtas']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
    
handler_for_this_call = CommandHandler('curtas', curtas)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def distanciamento(update, context):
    text = comunicado_header_to_section['🏡 Distanciamento social']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
handler_for_this_call = CommandHandler('distanciamento', distanciamento)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def pesquisa(update, context):
    text = comunicado_header_to_section['🔬 Pesquisa científica']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
    
handler_for_this_call = CommandHandler('pesquisa', pesquisa)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def saude(update, context):
    text = comunicado_header_to_section['Saúde Pública']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
    
handler_for_this_call = CommandHandler('saude', saude)
dispatcher.add_handler(handler_for_this_call)

In [ ]:
def ministerio(update, context):
    text = comunicado_header_to_section['🏥 Ministério da Saúde']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
    
handler_for_this_call = CommandHandler('ministerio', ministerio)
dispatcher.add_handler(handler_for_this_call)

In [25]:
def tratamento(update, context):
    text = comunicado_header_to_section['💊 Tratamento']

    if len(text) >4000:
        text_part_1, text_part_2 = split_parts(text)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_1)
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text_part_2)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=text)
    
handler_for_this_call = CommandHandler('tratamento', tratamento)
dispatcher.add_handler(handler_for_this_call)